In [ ]:
%%configure -f
{"executorMemory":"5G", "numExecutors":2, "executorCores":3, "conf":
{"spark.dynamicAllocation.enabled": "false"}}


In [1]:
val bucketname = "unibo-bd-antonelli2023"
val path_ml_tripdata =
"s3a://"+bucketname+"/datasets/fhvhv_tripdata_2021-01.parquet"
//"s3a://"+bucketname+"/datasets/project/2017/"
val path_ml_tripdata802 =
"s3a://"+bucketname+"/datasets/fhvhv_tripdata_2021-02.parquet"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1683036221120_0005,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd-antonelli2023
path_ml_tripdata: String = s3a://unibo-bd-antonelli2023/datasets/fhvhv_tripdata_2021-01.parquet
path_ml_tripdata802: String = s3a://unibo-bd-antonelli2023/datasets/fhvhv_tripdata_2021-02.parquet
res3: String = application_1683036221120_0005
res5: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1683036221120_0005/


In [8]:
import spark.implicits._
val parquetFileDF = spark.read.parquet(path_ml_tripdata)
val parquetFileDF802 = spark.read.parquet(path_ml_tripdata802)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import spark.implicits._
parquetFileDF: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]
parquetFileDF802: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]


In [59]:
import java.sql.Timestamp
case class TaxiTrip(
  license:String,
  request:Timestamp,
  pickup:Timestamp,
  dropoff:Timestamp,
  distance:Double,
  startloc:Long,
  //endloc:String,
  //time:Long,
  fare:Double,
)

object TaxiTrip {
    def extract(row:org.apache.spark.sql.Row) = {
        val license = row.getString(0)
        val request = row.getTimestamp(3)
        val pickup = row.getTimestamp(5)
        val dropoff = row.getTimestamp(6)
        val distance = row.getDouble(9)
        val startloc = row.getLong(7)
        //val endloc = row.getString(8)
        //val time = row.getLong(10)
        val fare = row.getDouble(11)
        
        new TaxiTrip(license,request,pickup,dropoff,distance,startloc,fare)
        

        //new TaxiTrip(license,request,pickup,dropoff,startloc,endloc,distance,time,fare)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.sql.Timestamp
defined class TaxiTrip
defined object TaxiTrip
Companions must be defined together; you may wish to use :paste mode for this.


In [60]:
import java.util.Calendar
def getDayTime(time: Long): Calendar = {
    var date:Calendar = Calendar.getInstance();
    date.setTimeInMillis(time);
    date
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.util.Calendar
getDayTime: (time: Long)java.util.Calendar


In [61]:
import org.apache.spark.HashPartitioner
val p = new HashPartitioner(12)
val data_rdd = parquetFileDF.rdd
val data_rdd802 = parquetFileDF802.rdd
val rddTaxiTrip = data_rdd.map(TaxiTrip.extract)
val rddTaxiTrip2 = data_rdd802.map(TaxiTrip.extract)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.HashPartitioner
p: org.apache.spark.HashPartitioner = org.apache.spark.HashPartitioner@c
data_rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[28] at rdd at <console>:37
data_rdd802: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[34] at rdd at <console>:37
rddTaxiTrip: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[70] at map at <console>:89
rddTaxiTrip2: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[71] at map at <console>:89


In [62]:
rddTaxiTrip.map(x => x.startloc).count()
//rddTaxiTrip.map(x => x.startloc).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res48: Long = 11908468


In [63]:
rddTaxiTrip.map(x => x.startloc).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res51: Long = 262


In [64]:
val query1 = rddTaxiTrip.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))
val query1802 = rddTaxiTrip2.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query1: org.apache.spark.rdd.RDD[(Double, Long)] = MapPartitionsRDD[77] at map at <console>:88
query1802: org.apache.spark.rdd.RDD[(Double, Long)] = MapPartitionsRDD[78] at map at <console>:88


In [65]:
val query1union = query1.union(query1802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query1union: org.apache.spark.rdd.RDD[(Double, Long)] = UnionRDD[79] at union at <console>:89


In [66]:
query1union.sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res52: Array[(Double, Long)] = Array((738.95,666), (527.11,575), (512.5,541), (480.73,512), (454.49,452), (432.359,379), (417.77,469), (408.19,440), (389.65,370), (381.95,444))


In [73]:
val query2 = rddTaxiTrip.map(x => (x.pickup.getTime() - x.request.getTime())/(60*1000))
val query2802 = rddTaxiTrip2.map(x => (x.pickup.getTime() - x.request.getTime())/(60*1000))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query2: org.apache.spark.rdd.RDD[Long] = MapPartitionsRDD[89] at map at <console>:88
query2802: org.apache.spark.rdd.RDD[Long] = MapPartitionsRDD[90] at map at <console>:88


In [75]:
//query2.collect()
val query2union = query2.union(query2802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query2union: org.apache.spark.rdd.RDD[Long] = UnionRDD[91] at union at <console>:90


In [78]:
query1union.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://localhost:8998/sessions/4/statements/78 with error payload: {"msg":"requirement failed: Session isn't active."}
